#### 女友內心：他是不是在想別的女人？
#### 我內心：每次走在路上，手上有垃圾卻找不到垃圾桶丟就覺得很煩躁，是不是有某個地方的垃圾桶密度很少？那他不就很孤單？

# 目標：找出大安區最孤單的公共垃圾桶

In [202]:
from IPython.display import clear_output
import time
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
import pandas as pd

%matplotlib inline

# 取得大安區公共垃圾桶資料

In [203]:
df = pd.read_csv('https://beta.data.taipei/api/getDatasetInfo/downloadResource?id=a835f3ba-7f50-4b0d-91a6-9df128632d1c&rid=f40cd66c-afba-4409-9289-e677b6b8d00e', encoding='big5')

In [204]:
df.head()

,序號,行政區,分隊別,路名,段、號及其他註明,經度,緯度,其他註明
0,1,大安區,敦化,仁愛路,4段449號,121.557360,25.037926,嚴禁投入家用垃圾，違者重罰新台幣6000元
1,2,大安區,敦化,仁愛路,4段403號,121.555656,25.037846,嚴禁投入家用垃圾，違者重罰新台幣6000元
2,3,大安區,敦化,仁愛路,4段387號,121.555129,25.037984,嚴禁投入家用垃圾，違者重罰新台幣6000元
3,4,大安區,敦化,仁愛路,4段325號,121.553102,25.038020,嚴禁投入家用垃圾，違者重罰新台幣6000元
4,5,大安區,敦化,仁愛路,4段325號,121.553135,25.037909,嚴禁投入家用垃圾，違者重罰新台幣6000元


## 整理出經緯度就好

In [205]:
df.columns[:]

Index(['序號', '行政區', '分隊別', '路名', '段、號及其他註明', '經度', '緯度', '其他註明'], dtype='object')

In [206]:
df1 = df[['經度','緯度']]

In [207]:
df1

,經度,緯度
0,121.557360,25.037926
1,121.555656,25.037846
2,121.555129,25.037984
3,121.553102,25.038020
4,121.553135,25.037909
5,121.551633,25.037942
6,121.550887,25.038067
7,121.548088,25.038411
8,121.547507,25.038009
9,121.546244,25.038164


## 有了經緯度，計算兩兩垃圾桶之間的距離

In [208]:
dist = []
dist_temp = []
R = 6371000           # 地球平均半徑(m)

for i in range(len(df1)):
    dist_temp.append(i)    # 加目前的編號到 dist_temp
    for j in range(len(df1)):
        dx = R*np.tan((3.14159265/180)*abs(df1.loc[i][0]-df1.loc[j][0]))   # 東西向距離 at little angle
        dy = R*np.tan((3.14159265/180)*abs(df1.loc[i][1]-df1.loc[j][1]))   # 南北向距離 at little angle
        dist_temp.append(((dx**2+dy**2)**0.5))    # 距離  
    dist.append(dist_temp)                      # 對於每個 i ，逐一把距離加到 dist 裡 
    dist_temp = []
print(dist[0][:15])      # 看一下第一組的前 15 個

[0, 0.0, 189.6848566663308, 248.15969932762602, 473.58335715368889, 469.80236836964934, 636.81583151760697, 719.93550272601749, 1032.408877990882, 1095.6424936045953, 1236.3260952005676, 1494.4380391460718, 1486.1081684708145, 516.51565954431692, 184.27066083142356]


## 所以說我們得到的 dist 的結構長這樣：

#### [   [編號0 , 和垃圾桶0的距離 , 和垃圾桶1的距離 , .....] , [編號1 , 和垃圾桶0的距離 , 和垃圾桶1的距離 , .....] , .........   ]

## 計算某個垃圾桶與其他垃圾桶的平均距離，記錄這當中平均距離最大的垃圾桶的編號

In [219]:
N = len(dist[0][1:])   # 「所有垃圾桶的距離」的總個數是固定的，設為 N
index = 0                # 編號儲存區
max_dist = 0           # 最大平均距離儲存區


for i in range(1,len(df1)):
    if sum(dist[i][1:])/N > sum(dist[index][1:])/N:
        if sum(dist[i][1:])/N > max_dist :
            index = i
            max_dist = sum(dist[i][1:])/N
print(index)

219


In [220]:
for i in range(10):
    print(len(dist[i][1:]))     # 檢查每組是否距離的個數都一樣

235
235
235
235
235
235
235
235
235
235


## 得到最孤單的垃圾桶

In [224]:
df[index:]   # 印出最孤單的垃圾桶的資訊


,序號,行政區,分隊別,路名,段、號及其他註明,經度,緯度,其他註明
219,220,大安區,瑞安,和平東路,二段181號,121.241967,25.025222,嚴禁投入家用垃圾，違者重罰新台幣6000元
220,221,大安區,瑞安,和平東路,二段133號,121.540830,25.025391,嚴禁投入家用垃圾，違者重罰新台幣6000元
221,222,大安區,瑞安,和平東路,二段(瑞安街口)(公車站牌),121.539077,25.025627,嚴禁投入家用垃圾，違者重罰新台幣6000元
222,223,大安區,瑞安,和平東路,二段63號(公車站牌),121.538680,25.025691,嚴禁投入家用垃圾，違者重罰新台幣6000元
223,224,大安區,瑞安,和平東路,二段49號,121.538127,25.025769,嚴禁投入家用垃圾，違者重罰新台幣6000元
224,225,大安區,瑞安,建國南路,二段117號對面(公車站牌),121.537450,25.029857,嚴禁投入家用垃圾，違者重罰新台幣6000元
225,226,大安區,瑞安,和平東路,二段(大安森林公園站牌),121.536437,25.025973,嚴禁投入家用垃圾，違者重罰新台幣6000元
226,227,大安區,瑞安,新生南路,二段(金華街口公車道),121.533826,25.030407,嚴禁投入家用垃圾，違者重罰新台幣6000元
227,228,大安區,瑞安,新生南路,二段(信義路口公車道),121.532700,25.033330,嚴禁投入家用垃圾，違者重罰新台幣6000元
228,229,大安區,瑞安,新生南路,二段(信義路口大安森林公園旁),121.532939,25.033329,嚴禁投入家用垃圾，違者重罰新台幣6000元


In [226]:
print(max_dist)
print(dist[218])
print(dist[219])

33264.864328
[218, 2149.0612907479967, 2005.5527534694504, 1975.9069374991873, 1832.1258980211692, 1824.6857486691908, 1729.1741681579424, 1696.1902677684454, 1593.0269912250064, 1528.8134070662095, 1505.017029469207, 1453.8977394069725, 1469.0720169163478, 2536.6729242012361, 2285.9926806627427, 1747.2087970399004, 1833.3383273558773, 1897.0378090785921, 1990.6275095093254, 2115.3255251273399, 2168.1581681679527, 2320.9346450035646, 2351.3330542143003, 2182.4781893327217, 2120.1001416399004, 2013.9774152706564, 1899.8256407322031, 1865.4473140974565, 2097.5204432014962, 1723.7213195083177, 1998.3291723821101, 2029.7798668363403, 1873.3318992715876, 1681.2139887457483, 1680.6584309734917, 2013.9774152706564, 1899.5914865377356, 1839.6558071356856, 2110.1270408069981, 1925.1244376698919, 1833.5168872524152, 1861.9718605165917, 1861.5413342222953, 1879.5994416074991, 1914.4087946123063, 2111.3991877817793, 2174.5791806290349, 1794.2440611336419, 1704.5226150852607, 1737.4610022525424, 16

### 啊...意外發現原始資料有誤呢